https://pythonhosted.org/pyDOE/factorial.html
    
## Factorial Designs

In this section, the following kinds of factorial designs will be described:

- General Full-Factorial

- 2-Level Full-Factorial

- 2-Level Fractional-Factorial

- Plackett-Burman

>Hint: All available designs can be accessed after a simple import statement:

In [2]:
# pip install --upgrade pyDOE2

  Created wheel for pyDOE2: filename=pyDOE2-1.3.0-py3-none-any.whl size=25518 sha256=81feeb14e25d47badce234dd401722b756edb98da9543d30d4b38d66449dc389
  Stored in directory: /home/tdird/.cache/pip/wheels/a7/de/26/98f9a9ff6082bdd6f2c003c6471f0fa7536b1ddd24dc2997e9
Successfully built pyDOE2
Note: you may need to restart the kernel to use updated packages.


In [2]:
# from pyDOE import *
from pyDOE2 import *
import numpy as np
import pandas as pd

## General Full-Factorial (fullfact)

This kind of design offers full flexibility as to the number of discrete levels for each factor in the design. Its usage is simple:

```
fullfact(levels)
```

where levels is an array of integers, like:

In [23]:
m1 = fullfact([2, 3])
m1

array([[0., 0.],
       [1., 0.],
       [0., 1.],
       [1., 1.],
       [0., 2.],
       [1., 2.]])

In [24]:
np.corrcoef(m1.T)

array([[1., 0.],
       [0., 1.]])

As can be seen in the output, the design matrix has as many columns as items in the input array.

## 2-Level Full-Factorial (`ff2n`)

This function is a convenience wrapper to fullfact that forces all the factors to have two levels each, you simple tell it how many factors to create a design for:

In [25]:
m2 = ff2n(3)
m2

array([[-1., -1., -1.],
       [ 1., -1., -1.],
       [-1.,  1., -1.],
       [ 1.,  1., -1.],
       [-1., -1.,  1.],
       [ 1., -1.,  1.],
       [-1.,  1.,  1.],
       [ 1.,  1.,  1.]])

In [26]:
np.corrcoef(m2.T)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

## 2-Level Fractional-Factorial (`fracfact`)

This function requires a little more knowledge of how the confounding will be allowed (this means that some factor effects get muddled with other interaction effects, so it’s harder to distinguish between them).

Let’s assume that we just can’t afford (for whatever reason) the number of runs in a full-factorial design. We can systematically decide on a fraction of the full-factorial by allowing some of the factor main effects to be confounded with other factor interaction effects. This is done by defining an alias structure that defines, symbolically, these interactions. These alias structures are written like “C = AB” or “I = ABC”, or “AB = CD”, etc. These define how one column is related to the others.

For example, the alias “C = AB” or “I = ABC” indicate that there are three factors (A, B, and C) and that the main effect of factor C is confounded with the interaction effect of the product AB, and by extension, A is confounded with BC and B is confounded with AC. A full- factorial design with these three factors results in a design matrix with 8 runs, but we will assume that we can only afford 4 of those runs. To create this fractional design, we need a matrix with three columns, one for A, B, and C, only now where the levels in the C column is created by the product of the A and B columns.

The input to fracfact is a generator string of symbolic characters (lowercase or uppercase, but not both) separated by spaces, like:

In [6]:
gen = 'a b ab'

This design would result in a 3-column matrix, where the third column is implicitly defined as "c = ab". This means that the factor in the third column is confounded with the interaction of the factors in the first two columns. The design ends up looking like this:

In [27]:
# fracfact('a b ab')
m3 = fracfact(gen)
m3

array([[-1., -1.,  1.],
       [ 1., -1., -1.],
       [-1.,  1., -1.],
       [ 1.,  1.,  1.]])

In [28]:
np.corrcoef(m3.T)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Fractional factorial designs are usually specified using the notation 2^(k-p), where k is the number of columns and p is the number of effects that are confounded. In terms of resolution level, higher is “better”. The above design would be considered a 2^(3-1) fractional factorial design, a 1/2-fraction design, or a Resolution III design (since the smallest alias “I=ABC” has three terms on the right-hand side). Another common design is a Resolution III, 2^(7-4) fractional factorial and would be created using the following string generator:

In [29]:
m4 = fracfact('a b ab c ac bc abc')
m4

array([[-1., -1.,  1., -1.,  1.,  1., -1.],
       [ 1., -1., -1., -1., -1.,  1.,  1.],
       [-1.,  1., -1., -1.,  1., -1.,  1.],
       [ 1.,  1.,  1., -1., -1., -1., -1.],
       [-1., -1.,  1.,  1., -1., -1.,  1.],
       [ 1., -1., -1.,  1.,  1., -1., -1.],
       [-1.,  1., -1.,  1., -1.,  1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [30]:
np.corrcoef(m4.T)

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

More sophisticated generator strings can be created using the “+” and “-” operators. The “-” operator swaps the levels of that column like this:

In [31]:
m5 = fracfact('a b -ab')
m5

array([[-1., -1., -1.],
       [ 1., -1.,  1.],
       [-1.,  1.,  1.],
       [ 1.,  1., -1.]])

In [32]:
np.corrcoef(m5.T)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In order to reduce confounding, we can utilize the fold function:

In [33]:
m = fracfact('a b ab')
m6 = fold(m)
m6

array([[-1., -1.,  1.],
       [ 1., -1., -1.],
       [-1.,  1., -1.],
       [ 1.,  1.,  1.],
       [ 1.,  1., -1.],
       [-1.,  1.,  1.],
       [ 1., -1.,  1.],
       [-1., -1., -1.]])

In [34]:
np.corrcoef(m6.T)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Applying the fold to all columns in the design breaks the alias chains between every main factor and two-factor interactions. This means that we can then estimate all the main effects clear of any two-factor interactions. Typically, when all columns are folded, this “upgrades” the resolution of the design.

By default, fold applies the level swapping to all columns, but we can fold specific columns (first column = 0), if desired, by supplying an array to the keyword columns:

In [35]:
m7 = fold(m, columns=[2])
m7

array([[-1., -1.,  1.],
       [ 1., -1., -1.],
       [-1.,  1., -1.],
       [ 1.,  1.,  1.],
       [-1., -1., -1.],
       [ 1., -1.,  1.],
       [-1.,  1.,  1.],
       [ 1.,  1., -1.]])

In [36]:
np.corrcoef(m7.T)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

>Note: Care should be taken to decide the appropriate alias structure for your design and the effects that folding has on it.

## Plackett-Burman (`pbdesign`)

Another way to generate fractional-factorial designs is through the use of **Plackett-Burman** designs. These designs are unique in that the number of trial conditions (rows) expands by multiples of four (e.g. 4, 8, 12, etc.). The max number of columns allowed before a design increases the number of rows is always one less than the next higher multiple of four.

For example, I can use up to 3 factors in a design with 4 rows:

In [37]:
m8 = pbdesign(3)
m8

array([[-1., -1.,  1.],
       [ 1., -1., -1.],
       [-1.,  1., -1.],
       [ 1.,  1.,  1.]])

In [38]:
np.corrcoef(m8.T)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

But if I want to do 4 factors, the design needs to increase the number of rows up to the next multiple of four (8 in this case):

In [39]:
m9 = pbdesign(4)
m9

array([[-1., -1.,  1., -1.],
       [ 1., -1., -1., -1.],
       [-1.,  1., -1., -1.],
       [ 1.,  1.,  1., -1.],
       [-1., -1.,  1.,  1.],
       [ 1., -1., -1.,  1.],
       [-1.,  1., -1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [40]:
np.corrcoef(m9.T)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

Thus, an 8-run Plackett-Burman design can handle up to (8 - 1) = 7 factors.

As a side note, It just so happens that the Plackett-Burman and 2^(7-4) fractional factorial design are identical:

In [41]:
m10 = pbdesign(7)
m10

array([[-1., -1.,  1., -1.,  1.,  1., -1.],
       [ 1., -1., -1., -1., -1.,  1.,  1.],
       [-1.,  1., -1., -1.,  1., -1.,  1.],
       [ 1.,  1.,  1., -1., -1., -1., -1.],
       [-1., -1.,  1.,  1., -1., -1.,  1.],
       [ 1., -1., -1.,  1.,  1., -1., -1.],
       [-1.,  1., -1.,  1., -1.,  1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [42]:
np.corrcoef(m10.T)

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [43]:
m11 = fracfact('a b ab c ac bc abc')
m11

array([[-1., -1.,  1., -1.,  1.,  1., -1.],
       [ 1., -1., -1., -1., -1.,  1.,  1.],
       [-1.,  1., -1., -1.,  1., -1.,  1.],
       [ 1.,  1.,  1., -1., -1., -1., -1.],
       [-1., -1.,  1.,  1., -1., -1.,  1.],
       [ 1., -1., -1.,  1.,  1., -1., -1.],
       [-1.,  1., -1.,  1., -1.,  1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [44]:
np.corrcoef(m11.T)

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [45]:
# np.all(pbdesign(7)==fracfact('a b ab c ac bc abc'))
np.all(m10 == m11)

True

## More Information

If the user needs more information about appropriate designs, please consult the following articles on Wikipedia:

- Factorial designs
    + http://en.wikipedia.org/wiki/Factorial_experiment

- Plackett-Burman designs
    + http://en.wikipedia.org/wiki/Plackett-Burman_design

There is also a wealth of information on the [NIST website](http://www.itl.nist.gov/div898/handbook/pri/pri.htm) about the various design matrices that can be created as well as detailed information about designing/setting-up/running experiments in general.

Any questions, comments, bug-fixes, etc. can be forwarded to the [author](tisimst@gmail.com).

## Create DOE for 2^(6-1) with minimal confounding

http://reliawiki.org/index.php/Two_Level_Factorial_Experiments

In [4]:
doe_m = ff2n(6)
doe_m.shape

(64, 6)

In [5]:
doe_df = pd.DataFrame(doe_m)
doe_df.shape

(64, 6)

In [6]:
doe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       64 non-null     float64
 1   1       64 non-null     float64
 2   2       64 non-null     float64
 3   3       64 non-null     float64
 4   4       64 non-null     float64
 5   5       64 non-null     float64
dtypes: float64(6)
memory usage: 3.1 KB


In [7]:
doe_df.columns = ['A', 'B', 'C', 'D', 'E', 'F']
doe_df.head()

,A,B,C,D,E,F
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
3,1.0,1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,1.0,-1.0,-1.0,-1.0


In [8]:
doe_df.corr()

,A,B,C,D,E,F
A,1.0,0.0,0.0,0.0,0.0,0.0
B,0.0,1.0,0.0,0.0,0.0,0.0
C,0.0,0.0,1.0,0.0,0.0,0.0
D,0.0,0.0,0.0,1.0,0.0,0.0
E,0.0,0.0,0.0,0.0,1.0,0.0
F,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
doe_df['AB'] = doe_df['A'] * doe_df['B']
doe_df['AC'] = doe_df['A'] * doe_df['C']
doe_df['AD'] = doe_df['A'] * doe_df['D']
doe_df['AE'] = doe_df['A'] * doe_df['E']
doe_df['AF'] = doe_df['A'] * doe_df['F']

doe_df['BC'] = doe_df['B'] * doe_df['C']
doe_df['BD'] = doe_df['B'] * doe_df['D']
doe_df['BE'] = doe_df['B'] * doe_df['E']
doe_df['BF'] = doe_df['B'] * doe_df['F']

doe_df['CD'] = doe_df['C'] * doe_df['D']
doe_df['CE'] = doe_df['C'] * doe_df['E']
doe_df['CF'] = doe_df['C'] * doe_df['F']

doe_df['DE'] = doe_df['D'] * doe_df['E']
doe_df['DF'] = doe_df['D'] * doe_df['F']

doe_df['EF'] = doe_df['E'] * doe_df['F']

In [10]:
doe_df.corr()

,A,B,C,D,E,F,AB,AC,AD,AE,...,BC,BD,BE,BF,CD,CE,CF,DE,DF,EF
A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AB,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
doe_df12 = doe_df[doe_df['A']*doe_df['B']*doe_df['C']*doe_df['D']*doe_df['E']*doe_df['F'] == 1]
doe_df12.shape

(32, 21)

In [13]:
doe_df22 = doe_df[doe_df['A']*doe_df['B']*doe_df['C']*doe_df['D']*doe_df['E']*doe_df['F'] == -1]
doe_df22.shape

(32, 21)

Checks

In [14]:
doe_df12[['AB', 'AC']].corr().iloc[0, 1]

0.0

In [15]:
main_effects = ['A', 'B', 'C', 'D', 'E', 'F']
two_way = []
for term1 in main_effects:
    for term2 in main_effects:
        if term1 < term2:
            two_way.append(term1 + term2)
two_way

['AB',
 'AC',
 'AD',
 'AE',
 'AF',
 'BC',
 'BD',
 'BE',
 'BF',
 'CD',
 'CE',
 'CF',
 'DE',
 'DF',
 'EF']

In [16]:
len(two_way)

15

In [18]:
doe_df12[[two_way[0], two_way[1]]].corr().iloc[0, 1]

0.0

In [19]:
def check_two_way(df):
    ctr = 1
    for i in range(0, len(two_way)):
        for j in range(i, len(two_way)):
            if two_way[i] != two_way[j]:
                r = df[[two_way[i], two_way[j]]].corr().iloc[0, 1]
                if r != 0:
                    print(ctr, " ", two_way[i], "-", two_way[j], ": ", r)
                    ctr += 1

In [21]:
check_two_way(doe_df12)

In [22]:
check_two_way(doe_df22)

In [23]:
doe_df12.corr()

,A,B,C,D,E,F,AB,AC,AD,AE,...,BC,BD,BE,BF,CD,CE,CF,DE,DF,EF
A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AB,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
doe_df22.corr()

,A,B,C,D,E,F,AB,AC,AD,AE,...,BC,BD,BE,BF,CD,CE,CF,DE,DF,EF
A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AB,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
doe_df12.head()

,A,B,C,D,E,F,AB,AC,AD,AE,...,BC,BD,BE,BF,CD,CE,CF,DE,DF,EF
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,...,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0
6,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0
9,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,...,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0


In [26]:
doe_df22.head()

,A,B,C,D,E,F,AB,AC,AD,AE,...,BC,BD,BE,BF,CD,CE,CF,DE,DF,EF
1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,...,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0
8,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0


In [37]:
use_doe = doe_df12[['A', 'B', 'C', 'D', 'E', 'F']]
use_doe

,A,B,C,D,E,F
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,1.0,1.0,-1.0,-1.0,-1.0,-1.0
5,1.0,-1.0,1.0,-1.0,-1.0,-1.0
6,-1.0,1.0,1.0,-1.0,-1.0,-1.0
9,1.0,-1.0,-1.0,1.0,-1.0,-1.0
10,-1.0,1.0,-1.0,1.0,-1.0,-1.0
12,-1.0,-1.0,1.0,1.0,-1.0,-1.0
15,1.0,1.0,1.0,1.0,-1.0,-1.0
17,1.0,-1.0,-1.0,-1.0,1.0,-1.0
18,-1.0,1.0,-1.0,-1.0,1.0,-1.0


In [38]:
use_doe2 = use_doe.append(pd.Series([0, 0, 0, 0, 0, 0], index=['A', 'B', 'C', 'D', 'E', 'F']), ignore_index=True)

In [39]:
use_doe2

,A,B,C,D,E,F
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1.0,1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,1.0,-1.0,-1.0,-1.0
3,-1.0,1.0,1.0,-1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,1.0,-1.0,-1.0
5,-1.0,1.0,-1.0,1.0,-1.0,-1.0
6,-1.0,-1.0,1.0,1.0,-1.0,-1.0
7,1.0,1.0,1.0,1.0,-1.0,-1.0
8,1.0,-1.0,-1.0,-1.0,1.0,-1.0
9,-1.0,1.0,-1.0,-1.0,1.0,-1.0


Parameter	-1	0	1
nar	        25	50	75
mni	       250	500	750
ps	       100	150	200
mp	       0.7	0.8	0.9
cp	      0.25	0.5	0.75
pp	      0.5	0.7	0.9

In [40]:
new_names = ['nar', 'mni', 'ps', 'mp', 'cp', 'pp']

dict_nar = {-1: 25, 0: 50, 1: 75}
dict_mni = {-1: 250, 0: 500, 1: 750}
dict_ps = {-1: 100, 0: 150, 1: 200}
dict_mp = {-1: 0.7, 0: 0.8, 1: 0.9}
dict_cp = {-1: 0.25, 0: 0.5, 1: 0.75}
dict_pp = {-1: 0.5, 0: 0.7, 1: 0.9}
        
use_doe3 = use_doe2.copy()
use_doe3.columns = new_names

use_doe3

,nar,mni,ps,mp,cp,pp
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1.0,1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,1.0,-1.0,-1.0,-1.0
3,-1.0,1.0,1.0,-1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,1.0,-1.0,-1.0
5,-1.0,1.0,-1.0,1.0,-1.0,-1.0
6,-1.0,-1.0,1.0,1.0,-1.0,-1.0
7,1.0,1.0,1.0,1.0,-1.0,-1.0
8,1.0,-1.0,-1.0,-1.0,1.0,-1.0
9,-1.0,1.0,-1.0,-1.0,1.0,-1.0


In [41]:
use_doe3['nar'] = use_doe3['nar'].map(dict_nar)
use_doe3['mni'] = use_doe3['mni'].map(dict_mni) 
use_doe3['ps'] = use_doe3['ps'].map(dict_ps) 
use_doe3['mp'] = use_doe3['mp'].map(dict_mp) 
use_doe3['cp'] = use_doe3['cp'].map(dict_cp)
use_doe3['pp'] = use_doe3['pp'].map(dict_pp) 

In [42]:
use_doe3

,nar,mni,ps,mp,cp,pp
0,25,250,100,0.7,0.25,0.5
1,75,750,100,0.7,0.25,0.5
2,75,250,200,0.7,0.25,0.5
3,25,750,200,0.7,0.25,0.5
4,75,250,100,0.9,0.25,0.5
5,25,750,100,0.9,0.25,0.5
6,25,250,200,0.9,0.25,0.5
7,75,750,200,0.9,0.25,0.5
8,75,250,100,0.7,0.75,0.5
9,25,750,100,0.7,0.75,0.5


In [43]:
use_doe3.to_csv('hpt_doe.csv', index=False)